# Part 1: Environment Setup
1. Install Spark + Java in Google Colab.
2. Initialize Spark with app name "ProductSalesAnalysis"

In [4]:
# Download Spark from the correct URL
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz


--2025-07-30 12:20:44--  https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz’

spark-3.5.0-bin-had 100%[===================>] 381.85M  19.9MB/s    in 60s     

2025-07-30 12:21:45 (6.34 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz’ saved [400395283/400395283]



In [5]:
# Extract Spark
!tar -xzf spark-3.5.0-bin-hadoop3.tgz

In [6]:
# Install Java & Findspark
!apt-get install openjdk-11-jdk -y
!pip install -q findspark

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,367 kB of archives.
After this operation, 15.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security/main amd64 openjdk-11-jre amd64 11.0.28+6-1ubuntu1~22.04.1 [214 kB]
Get:3 http://security.

In [7]:
# Set Environment Variables and Initialize Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ProductSalesAnalysis").getOrCreate()
spark.version

'3.5.0'

 # Part 2: Load Sales Data from CSV

In [8]:
# Create the sales.csv file
csv_data = """OrderID,Product,Category,Quantity,UnitPrice,Region
1001,Mobile,Electronics,2,15000,North
1002,Laptop,Electronics,1,55000,South
1003,T-Shirt,Apparel,3,500,East
1004,Jeans,Apparel,2,1200,North
1005,TV,Electronics,1,40000,West
1006,Shoes,Footwear,4,2000,South
1007,Watch,Accessories,2,3000,East
1008,Headphones,Electronics,3,2500,North"""

with open("sales.csv", "w") as file:
    file.write(csv_data)

In [9]:
# Read CSV into PySpark DataFrame
df = spark.read.csv("sales.csv", header=True, inferSchema=True)

In [10]:
# Print Schema and Show Top 5 Rows
df.printSchema()
df.show(5)

root
 |-- OrderID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: integer (nullable = true)
 |-- Region: string (nullable = true)

+-------+-------+-----------+--------+---------+------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|
+-------+-------+-----------+--------+---------+------+
|   1001| Mobile|Electronics|       2|    15000| North|
|   1002| Laptop|Electronics|       1|    55000| South|
|   1003|T-Shirt|    Apparel|       3|      500|  East|
|   1004|  Jeans|    Apparel|       2|     1200| North|
|   1005|     TV|Electronics|       1|    40000|  West|
+-------+-------+-----------+--------+---------+------+
only showing top 5 rows



# Part -3 Business Questions
1. Add a new column TotalPrice = Quantity × UnitPrice
2. Total revenue generated across all regions.
3. Category-wise revenue sorted in descending order.
4. Region with the highest number of orders
5. Average Unit Price per Category
6. All orders where TotalPrice is more than
30,000

In [11]:
# 1. Add a new column TotalPrice = Quantity × UnitPrice
from pyspark.sql.functions import col

df = df.withColumn("TotalPrice", col("Quantity") * col("UnitPrice"))
df.show()

+-------+----------+-----------+--------+---------+------+----------+
|OrderID|   Product|   Category|Quantity|UnitPrice|Region|TotalPrice|
+-------+----------+-----------+--------+---------+------+----------+
|   1001|    Mobile|Electronics|       2|    15000| North|     30000|
|   1002|    Laptop|Electronics|       1|    55000| South|     55000|
|   1003|   T-Shirt|    Apparel|       3|      500|  East|      1500|
|   1004|     Jeans|    Apparel|       2|     1200| North|      2400|
|   1005|        TV|Electronics|       1|    40000|  West|     40000|
|   1006|     Shoes|   Footwear|       4|     2000| South|      8000|
|   1007|     Watch|Accessories|       2|     3000|  East|      6000|
|   1008|Headphones|Electronics|       3|     2500| North|      7500|
+-------+----------+-----------+--------+---------+------+----------+



In [12]:
# 2. Total revenue generated across all regions.
from pyspark.sql.functions import sum


df.agg(sum("TotalPrice").alias("TotalRevenue")).show()

+------------+
|TotalRevenue|
+------------+
|      150400|
+------------+



In [13]:
# 3. Category-wise revenue sorted in descending order.
from pyspark.sql.functions import desc

category_revenue = df.groupBy("Category").agg(sum("TotalPrice").alias("TotalRevenue"))
category_revenue = category_revenue.orderBy(desc("TotalRevenue"))
category_revenue.show()

+-----------+------------+
|   Category|TotalRevenue|
+-----------+------------+
|Electronics|      132500|
|   Footwear|        8000|
|Accessories|        6000|
|    Apparel|        3900|
+-----------+------------+



In [14]:
# 4. Region with the highest number of orders
from pyspark.sql.functions import desc

region_order_count = df.groupBy("Region").count()
region_order_count = region_order_count.orderBy(desc("count"))
region_order_count.show(1)

+------+-----+
|Region|count|
+------+-----+
| North|    3|
+------+-----+
only showing top 1 row



In [15]:
# 5. Average Unit Price per Category
from pyspark.sql.functions import avg

average_price_per_category = df.groupBy("Category").agg(avg("UnitPrice").alias("AverageUnitPrice"))
average_price_per_category.show()

+-----------+----------------+
|   Category|AverageUnitPrice|
+-----------+----------------+
|    Apparel|           850.0|
|Electronics|         28125.0|
|   Footwear|          2000.0|
|Accessories|          3000.0|
+-----------+----------------+



In [16]:
# 6. All orders where TotalPrice is more than 30,000
filtered_orders = df.filter(col("TotalPrice") > 30000)
filtered_orders.show()


+-------+-------+-----------+--------+---------+------+----------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|TotalPrice|
+-------+-------+-----------+--------+---------+------+----------+
|   1002| Laptop|Electronics|       1|    55000| South|     55000|
|   1005|     TV|Electronics|       1|    40000|  West|     40000|
+-------+-------+-----------+--------+---------+------+----------+



# Part 4Data Transformations
1. Create a new column HighValueOrder which is "Yes" if TotalPrice > 20,000,
else "No" .
2. Filter and display all high-value orders in the North region.
3. Count how many high-value orders exist per region.

In [23]:
from pyspark.sql.functions import when
from pyspark.sql.functions import col, when, count


# 1. HighValueOrder column
df = df.withColumn("HighValueOrder", when(col("TotalPrice") > 20000, "Yes").otherwise("No"))

df.show()

+-------+----------+-----------+--------+---------+------+----------+--------------+
|OrderID|   Product|   Category|Quantity|UnitPrice|Region|TotalPrice|HighValueOrder|
+-------+----------+-----------+--------+---------+------+----------+--------------+
|   1001|    Mobile|Electronics|       2|    15000| North|     30000|           Yes|
|   1002|    Laptop|Electronics|       1|    55000| South|     55000|           Yes|
|   1003|   T-Shirt|    Apparel|       3|      500|  East|      1500|            No|
|   1004|     Jeans|    Apparel|       2|     1200| North|      2400|            No|
|   1005|        TV|Electronics|       1|    40000|  West|     40000|           Yes|
|   1006|     Shoes|   Footwear|       4|     2000| South|      8000|            No|
|   1007|     Watch|Accessories|       2|     3000|  East|      6000|            No|
|   1008|Headphones|Electronics|       3|     2500| North|      7500|            No|
+-------+----------+-----------+--------+---------+------+-------

In [21]:
# 2. High-value orders in North region
df.filter((col("HighValueOrder") == "Yes") & (col("Region") == "North")).show()

+-------+-------+-----------+--------+---------+------+----------+--------------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|TotalPrice|HighValueOrder|
+-------+-------+-----------+--------+---------+------+----------+--------------+
|   1001| Mobile|Electronics|       2|    15000| North|     30000|           Yes|
+-------+-------+-----------+--------+---------+------+----------+--------------+



In [22]:
# 3. Count high-value orders per region
df.filter(col("HighValueOrder") == "Yes") \
  .groupBy("Region") \
  .agg(count("OrderID").alias("HighValueOrderCount")) \
  .show()

+------+-------------------+
|Region|HighValueOrderCount|
+------+-------------------+
| South|                  1|
|  West|                  1|
| North|                  1|
+------+-------------------+



# Part 5: Save Results
Save the transformed DataFrame as a CSV file named high_value_orders.csv with
headers.

In [24]:
# Save high-value orders to CSV
df.filter(col("HighValueOrder") == "Yes") \
  .write.csv("high_value_orders.csv", header=True, mode="overwrite")

# View saved folder in Colab
!ls high_value_orders.csv


part-00000-71969407-b470-4ca3-8bb6-49c518991083-c000.csv  _SUCCESS
